<a href="https://colab.research.google.com/github/pravin691983/MelanomaDetectionAssignment/blob/main/PRAVIN_TAWADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello  World")